In [ ]:
import tensorflow

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 693366051319316151
 xla_global_id: -1]

In [8]:
from tqdm import tqdm
import numpy as np
import random
from collections import deque

class Edge:
    def __init__(self, node1, node2, relationship):
        self.node1 = node1
        self.node2 = node2
        self.relationship = relationship
    def __eq__(self, other):
        return self.node1.value == other.node1.value and self.node2.value == other.node2.value and self.relationship.name == other.relationship.name 
    def __hash__(self):
        return hash(self.node1.value + self.node2.value + self.relationship.name) 
    def __str__(self):
        return f"{str(self.node1)} {str(self.relationship)}  {str(self.node2)}"
class Relationship:
    """Hashable relationship with its name as ID"""
    def __init__(self, name):
        self.name = name.strip()
    def __eq__(self, n1):
        """Relationships are equal if they have the same name"""
        return self.name == n1.name 
    def __str__(self):
        return self.name
    def __hash__(self):
        """Relationship name identifies the relationship"""
        return hash(self.name) 
class Node:
    """Hashable node with its value as ID"""
    def __init__(self, value):
        self.value = value.strip()
    def __str__(self):
        return self.value
    """Nodes are equal if they have the same value"""
    def __eq__(self, n1):
        return self.value == n1.value
    """Node value identifies the node"""
    def __hash__(self):
        return hash(self.value) 
class Graph:
    """Graph stucture"""
    def __init__(self):
        """Initialize the graph"""
        #sets because we need efficient lookups (O(1) for set O(n) for list)
        self.nodes = set()
        self.relationships = set()
        self.edges = []
        self.node2id = {}
        self.rel2id = {}
    def add_triple(self, s_val, o_val, p_val):
        """Add a single triple to the graph."""
        #set is comprised of unique element, adding an existing element doesn't affect the set 
        self.nodes.add(s_val)
        self.nodes.add(o_val)
        self.relationships.add(p_val)
        self.edges.append(Edge(s_val, o_val, p_val))

    def finalize(self):
        """Build indices once after all data is loaded."""
        self.node2id = {node: i for i, node in enumerate(self.nodes)}
        self.rel2id = {rel: i for i, rel in enumerate(self.relationships)}
        self.adj = {node: [] for node in self.nodes}
        for e in self.edges:
            self.adj[e.node1].append((e.node2, e.relationship))
    def find_l1_paths(self, node):
        pass
    def paths2dataset(self):
        pass      
    def find_paths(self, s, t):
        """Optimized iterative path finding with max depth 6. This function is slow for our problem, moving implementation to C++ with paralellization."""
        all_paths = []
        # Queue stores: (current_node, current_path, visited_set)
        queue = deque([(s, [(s, None)], {s})])
        
        while queue:
            curr_node, path, visited = queue.popleft()
            
            # Stop if path exceeds max length
            if len(path) > 6:
                continue
            # if len(all_paths) == 3:
            #     continue
            for neighbor, rel in self.adj.get(curr_node, []):
                if neighbor == t:
                    # Path length must be > 1 (more than 2 nodes in list)
                    if len(path) > 2:
                        all_paths.append(path + [(neighbor, rel)])
                    continue # Found target, don't need to go deeper from here (simple path)
                
                if neighbor not in visited and len(path) < 4:
                    # Use set union for efficiency in creating the next visited set
                    queue.append((neighbor, path + [(neighbor, rel)], visited | {neighbor}))
        return all_paths
    
    def graph2dataset(self):
        """Convert data to numpy arrays compatible as neural net input"""
        pos_triples = np.array([
            [self.node2id[e.node1], self.node2id[e.node2], self.rel2id[e.relationship]] 
            for e in self.edges
        ], dtype=np.int32)
        num_pos = len(pos_triples)
        num_nodes = len(self.nodes)
        num_rels = len(self.relationships)
        # We over-sample by 10% to account for accidental "real" edges being picked
        oversample_factor = 1.1
        num_to_sample = int(num_pos * oversample_factor)

        neg_subs = np.random.randint(0, num_nodes, num_to_sample)
        neg_objs = np.random.randint(0, num_nodes, num_to_sample)
        neg_rels = np.random.randint(0, num_rels, num_to_sample)
        
        neg_triples = np.stack([neg_subs, neg_objs, neg_rels], axis=1)

        #Pruning: Filter out samples where sub == obj or triple exists in positive set

        def hash_triples(triples):
            # Maps (s, o, r) to a single unique integer
            return triples[:, 0] * (num_nodes * num_rels) + triples[:, 1] * num_rels + triples[:, 2]

        pos_hashes = set(hash_triples(pos_triples))
        neg_hashes = hash_triples(neg_triples)
        mask = np.array([(h not in pos_hashes) for h in neg_hashes])
        mask &= (neg_triples[:, 0] != neg_triples[:, 1])
        valid_negatives = neg_triples[mask][:num_pos]
        X = np.vstack([pos_triples, valid_negatives])
        y = np.concatenate([np.ones(num_pos), np.zeros(len(valid_negatives))])

        return X, y
            
g = Graph()  
l_bar = '{desc}: {percentage:.3f}%|'
r_bar = '| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, ' '{rate_fmt}{postfix}]'
format = '{l_bar}{bar}{r_bar}'
data = open('../../SiaILP/data/lineage/test.txt').readlines()  
for line in tqdm(data, ncols=100, bar_format=format):
    s,p,o = line.split('\t')
    g.add_triple(Node(s), Node(o), Relationship(p))
g.finalize()
X,y = g.graph2dataset()
file = open('triples.data','w+')
for edge in X:
    file.write(f"{edge[0]} {edge[1]} {edge[2]}\n")

file.close()

100%|███████████████████████████████████████████████████| 253990/253990 [00:00<00:00, 407119.52it/s]


In [58]:
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Dot, Normalization, Lambda, LSTM, Bidirectional, MaxPooling1D, Flatten, Reshape
from tensorflow.keras.models import Model



num_nodes = len(g.nodes) + 1 
num_rels = len(g.relationships) + 1

#single input for the triple [s, o, r]
triple_input = Input(shape=(3,), name="triple_input")
#slice the input
s_idx = Lambda(lambda x: x[:, 0])(triple_input)
o_idx = Lambda(lambda x: x[:, 1])(triple_input)
r_idx = Lambda(lambda x: x[:, 2])(triple_input)
#embedding Layers
node_emb_layer = Embedding(input_dim=num_nodes, output_dim=300, name="Node_Embedding")
rel_emb_layer = Embedding(input_dim=num_rels, output_dim=300, name="Rel_Embedding")
s_emb = node_emb_layer(s_idx)
o_emb = node_emb_layer(o_idx)
r_emb = rel_emb_layer(r_idx)
#reshape in order to fit LSTM
s_seq = Reshape((1, 300))(s_emb)
o_seq = Reshape((1, 300))(o_emb)
#add 2 layer bi-directional LSTM
lstm_layer_1 = Bidirectional(LSTM(150, return_sequences=True))
lstm_layer_2 = Bidirectional(LSTM(150, return_sequences=True))
#first LSTM layer
fst_lstm_mid = lstm_layer_1(s_seq)
scd_lstm_mid = lstm_layer_1(o_seq)
#second LSTM layer
fst_lstm_fin = lstm_layer_2(fst_lstm_mid)
scd_lstm_fin = lstm_layer_2(scd_lstm_mid)
#reduce max
pool_layer = MaxPooling1D(pool_size=1) 
fst_pooled = pool_layer(fst_lstm_fin)
scd_pooled = pool_layer(scd_lstm_fin)
#flatten
fst_final = Flatten()(fst_pooled)
scd_final = Flatten()(scd_pooled)
#merge node embeddings with DNN
nodes_concat = Concatenate()([s_emb, o_emb])
nodes_representation = Dense(300)(nodes_concat)
#normalization
rel_normalized = Normalization(axis=-1)(r_emb)
nodes_normalized = Normalization(axis=-1)(nodes_representation)
#edge probability
edge_probability = Dot(axes=-1)([rel_normalized, nodes_normalized])
edge_probability = Flatten()(edge_probability)
output = Dense(1, activation='sigmoid')(edge_probability)

m = Model(inputs = triple_input, outputs = output)
m.compile(loss='binary_crossentropy', metrics=['accuracy'])
m.fit(X,y,validation_split=0.1)



14287/14287 ━━━━━━━━━━━━━━━━━━━━ 609s 43ms/step - accuracy: 0.9421 - loss: 0.1400 - val_accuracy: 0.9703 - val_loss: 0.1075
